In [1]:
import pandas as pd
import pandas_ta as ta

In [3]:
test_df = pd.read_csv('/home/duypham/Mlops/Lab01/Data/CrawlBitCoin.csv')
test_df.set_index(pd.to_datetime(test_df['Date Time']),inplace=True)
test_df.drop(columns=['Date Time'],inplace=True)
test_df.dropna(inplace=True)
test_df

,Unnamed: 0,Open,High,Low,Close,Volume
Date Time,,,,,,
2025-04-08 00:24:00,0,78360.74,78457.83,78319.90,78396.37,46.324
2025-04-08 00:23:00,1,78300.00,78360.75,78240.00,78360.75,30.243
2025-04-08 00:22:00,2,78140.00,78315.99,78094.70,78299.96,24.673
2025-04-08 00:21:00,3,78332.00,78434.78,78094.69,78139.99,41.665
2025-04-08 00:20:00,4,78341.99,78437.30,78279.90,78331.99,49.813
...,...,...,...,...,...,...
2024-08-03 03:30:00,375746,62542.00,62659.97,62473.45,62618.01,114.744
2024-08-03 03:29:00,375747,62388.00,62636.69,62279.06,62542.01,106.961
2024-08-03 03:28:00,375748,62218.41,62388.01,62192.03,62388.00,70.196


In [2]:
cp_df= pd.read_csv('/home/duypham/Mlops/Lab01/Data/CrawlBitCoin.csv')
cp_df.set_index(pd.to_datetime(cp_df['Date Time']),inplace=True)
cp_df.dropna(inplace=True)
cp_df = cp_df[['Low','High','Close']]

cp_df=cp_df.resample('15min').agg({
'Low': 'min',
'High': 'max',
'Close': 'last'
}).dropna()[['Low','High','Close']]
cp_df['Target']=cp_df.Close.shift(-1)
cp_df.dropna(inplace=True)
cp_df

,Low,High,Close,Target
Date Time,,,,
2024-08-03 03:15:00,62088.00,62728.01,62542.01,62320.70
2024-08-03 03:30:00,62295.23,62682.00,62320.70,62622.01
2024-08-03 03:45:00,62268.01,62675.37,62622.01,61908.13
2024-08-03 04:00:00,61900.00,62736.01,61908.13,61660.86
2024-08-03 04:15:00,61650.00,62215.99,61660.86,62065.85
...,...,...,...,...
2025-04-09 15:30:00,77086.95,77341.21,77272.22,77298.71
2025-04-09 15:45:00,77121.79,77373.11,77298.71,77353.89
2025-04-09 16:00:00,77235.29,77449.74,77353.89,77244.00


In [6]:
cp_df.index.max()

Timestamp('2025-04-09 16:30:00')

In [3]:
new_df= cp_df.copy(deep=True)
rsi_feature=ta.rsi(new_df.Close)
atr_feature=ta.atr(new_df.High,new_df.Low,new_df.Close)
adx_feature=ta.adx(new_df.High,new_df.Low,new_df.Close)
sma_feature=ta.sma(new_df.Close)
skew_feature=ta.skew(new_df.Close)
slope_feature=ta.slope(new_df.Close)
bband_feature=ta.bbands(new_df.Close).iloc[:,[0,2]]
macd_feature=ta.macd(new_df.Close).iloc[:,[0,2]]

new_df=pd.concat([new_df.iloc[:,:-1],sma_feature,rsi_feature,atr_feature,adx_feature,skew_feature,slope_feature,bband_feature,macd_feature,new_df.iloc[:,-1]],axis=1)
new_df=new_df.dropna()

max_date = new_df.index.max()

test_start = max_date - pd.DateOffset(months=6)

train_df = new_df[new_df.index < test_start]
test_df = new_df[new_df.index >= test_start]

val_start = test_start - pd.DateOffset(months=3)

val_df = train_df[train_df.index >= val_start]
train_df = train_df[train_df.index < val_start]

In [4]:
train_df

,Low,High,Close,SMA_10,RSI_14,ATRr_14,ADX_14,DMP_14,DMN_14,SKEW_30,SLOPE_1,BBL_5_2.0,BBU_5_2.0,MACD_12_26_9,MACDs_12_26_9,Target
Date Time,,,,,,,,,,,,,,,,


In [9]:
from pandas.tseries.offsets import DateOffset
val_end = test_df.index.max()
val_start = val_end - DateOffset(months=6)

In [10]:
val_start

Timestamp('2024-10-09 16:49:00')

In [14]:
test_df[test_df.index<val_start].tail()

,Unnamed: 0,Open,High,Low,Close,Volume
Date Time,,,,,,
2024-08-03 03:30:00,375746,62542.00,62659.97,62473.45,62618.01,114.744
2024-08-03 03:29:00,375747,62388.00,62636.69,62279.06,62542.01,106.961
2024-08-03 03:28:00,375748,62218.41,62388.01,62192.03,62388.00,70.196
2024-08-03 03:27:00,375749,62693.99,62694.00,62088.00,62219.99,564.425
2024-08-03 03:26:00,375750,62702.00,62728.01,62660.00,62693.99,17.871
